## Causal Attention V1

- Sử dụng `SelfAttention_v2` được implement ở phần trước

In [1]:
import torch.nn as nn
import torch 

class SelfAttention_v2(nn.Module):
    def __init__(self, dim_in, dim_out, qkv_bias=False):
        super().__init__()     # Khởi tạo lớp cha nn.Module
        self.dim_out = dim_out  # Kích thước đầu ra
        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)  # Lớp Linear cho query
        self.W_key = nn.Linear(dim_in, dim_out, bias=qkv_bias)    # Lớp Linear cho key
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)  # Lớp Linear cho value

    def forward(self, x):
        queries = self.W_query(x)   # X . Wq 
        keys = self.W_key(x)         # X . Wk
        values = self.W_value(x)     # X . Wv

        attn_scores = queries @ keys.T      # Tính attention scores (omega)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vectors = attn_weights @ values  # Tính context vectors (z)

        return context_vectors

In [2]:
torch.manual_seed(123)
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)
self_attention_v2 = SelfAttention_v2(dim_in=3, dim_out=2, qkv_bias=False)
print(self_attention_v2(inputs))    # run forward with inputs

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


- Sử dụng `causal attention mask`, bao gồm 3 bước như sau:

    + 1. Áp dụng _softmax_.
    + 2. _Mask_ bằng 0 với các weights phía trên đường chéo chính.
    + 3. Normalize các hàng.

#### 1. Áp dụng softmax

In [12]:
queries = self_attention_v2.W_query(inputs)
keys = self_attention_v2.W_key(inputs)
attention_scores = queries @ keys.T
print("Attention Scores:")
print(attention_scores)  # print attention scores   
attention_weights = torch.softmax(
    attention_scores / keys.shape[-1]**0.5, dim=-1
)
print("\nAttention Weights:")
print(attention_weights)  # print attention weights

Attention Scores:
tensor([[0.3111, 0.3479, 0.3471, 0.1714, 0.2350, 0.1928],
        [0.1655, 0.2602, 0.2576, 0.1445, 0.1384, 0.1790],
        [0.1667, 0.2602, 0.2577, 0.1443, 0.1391, 0.1784],
        [0.0510, 0.1080, 0.1064, 0.0643, 0.0476, 0.0835],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121, 0.1174],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MmBackward0>)

Attention Weights:
tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


#### 2. _Mask_ với các weights phía trên đường chéo chính.


- Sử dụng `.tril()` của Pytorch cho mask:

In [ ]:
context_length = attention_scores.shape[0]  # length of the input text
mask_simple = torch.tril(torch.ones((context_length, context_length)))  # tril = Triangle Lower
print(mask_simple)  # print the simple mask

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- Áp dụng _mask_ này cho _attention weights_ bằng cách sử dụng `element-wise product`:

In [5]:
masked_simple = attention_weights * mask_simple
print(masked_simple)  # print the masked attention weights

tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MulBackward0>)


#### 3. Normalize các hàng.


- Sau khi áp dụng _mask_, ta cần thêm bước _normalize_ theo từng hàng để tổng từng hàng vẫn bằng 1 như cũ.

In [6]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)  # print the normalized masked attention weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)


- Khi ta áp dụng _mask_ & _renormalize_ lại _attention weights_, thoạt nhìn thì có vẻ các _future token_ vẫn đóng góp vào _token hiện tại_ vì phép tính _softmax_ tính toán trên từng hàng, nhưng thực tế không như vậy. 

- Vì khi ta _renormalize_ lại sau khi _mask_, bản chất ta đang tính _softmax_ trên `1 tập nhỏ hơn`, với các token bị loại là các `masked token`.

## Causal Attention V2

- Ở version 2 này ta chỉ cần _mask_ với 2 bước. Thay vì _mask_ các `future token` bằng 0, ta sẽ _mask_ bằng $-\infty$.

- Sử dụng `causal attention mask v2`, bao gồm 2 bước như sau:

    + 1. _Mask_ $-\infty$ với các scores phía trên đường chéo chính.
    + 2. Áp dụng _softmax_.

#### 1. _Mask_ $-\infty$ với các scores phía trên đường chéo chính.


In [17]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)   # triu = Triangle Upper
print("Triangle Upper Mask:")
print(mask)  # print the upper triangular mask

print("\nMasked Attention Scores:")
masked = attention_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

Triangle Upper Mask:
tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

Masked Attention Scores:
tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


#### 2. Áp dụng softmax

In [13]:
attention_weights = torch.softmax(
    masked / keys.shape[-1]**0.5, dim=-1
)

print(attention_weights)  # print the masked attention weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


In [15]:
context_vectors = attention_weights @ self_attention_v2.W_value(inputs)
context_vectors

tensor([[-0.4519,  0.2216],
        [-0.5874,  0.0058],
        [-0.6300, -0.0632],
        [-0.5675, -0.0843],
        [-0.5526, -0.0981],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)